<a href="https://colab.research.google.com/github/pnyokok/2024_IAPproject/blob/main/YOLOv7_tiny_ppdetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Google Drive datasets.zip
!cp /content/drive/MyDrive/data/datasets.zip .

!unzip datasets.zip


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: datasets/train/labels/photo-1557862921-37829c790f19_jpg.rf.5f8fac76501c934fcd70f4498d3a11fe.txt  
  inflating: datasets/train/labels/photo-1557862921-37829c790f19_jpg.rf.c95695bfb474af6a8e3fe56497990e00.txt  
  inflating: datasets/train/labels/photo-1557862921-37829c790f19_jpg.rf.e453f7f8033abf8f5b3b116266a10166.txt  
  inflating: datasets/train/labels/photo-1557953674-830fbda7a569_jpg.rf.3e85cd0aa953e286544643482015d317.txt  
  inflating: datasets/train/labels/photo-1557953674-830fbda7a569_jpg.rf.b4f2ba018c01790ca3186c8c61cf29a6.txt  
  inflating: datasets/train/labels/photo-1557953674-830fbda7a569_jpg.rf.d88f24fac86b38c1f4661155acb31579.txt  
  inflating: datasets/train/labels/photo-1558885544-2defc62e2e2b_jpg.rf.73b0b0d8bcc5cd19c0c2223a1657ae11.txt  
  inflating: datasets/train/labels/photo-1559443194-d7ca66cc2cc5_jpg.rf.004a9e1125df0e7d74bcbea2d8e298c5.txt  
  inflating: datasets/train/labels/photo-1559443194-d7ca66cc2cc5_jpg.rf.071a

In [ ]:
# YOLOv7
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7
!pip install -r requirements.txt


Cloning into 'yolov7'...
remote: Enumerating objects: 1197, done.
remote: Total 1197 (delta 0), reused 0 (delta 0), pack-reused 1197
Receiving objects: 100% (1197/1197), 74.23 MiB | 19.87 MiB/s, done.
Resolving deltas: 100% (519/519), done.
/content/yolov7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 46.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-m

# Set GPU

In [ ]:
import torch

# GPU 활성화 확인
print("CUDA available: ", torch.cuda.is_available())
print("CUDA device count: ", torch.cuda.device_count())
print("CUDA current device: ", torch.cuda.current_device())
print("CUDA device name: ", torch.cuda.get_device_name(torch.cuda.current_device()))


CUDA available:  True
CUDA device count:  1
CUDA current device:  0
CUDA device name:  Tesla T4


# Images Su-Jung

In [ ]:
import os

def check_dataset_images(image_dir, label_dir):
    missing_images = []

    for label_file in os.listdir(label_dir):
        if label_file.endswith('.txt'):
            image_file = label_file.replace('.txt', '.jpg')
            image_path = os.path.join(image_dir, image_file)
            if not os.path.exists(image_path):
                missing_images.append(image_path)

    return missing_images

image_dir = '/content/yolov7/datasets/train/images'  # 이미지 파일 경로
label_dir = '/content/yolov7/datasets/train/labels'  # 라벨 파일 경로

missing_images = check_dataset_images(image_dir, label_dir)
if missing_images:
    print("Missing image files:")
    for img in missing_images:
        print(img)
else:
    print("All image files are present.")


All image files are present.


In [ ]:
import os

def check_dataset_images(image_dir, label_dir):
    missing_images = []

    for label_file in os.listdir(label_dir):
        if label_file.endswith('.txt'):
            image_file = label_file.replace('.txt', '.jpg')
            image_path = os.path.join(image_dir, image_file)
            image_path = image_path.replace("\\", "/")  # 경로 구분자 변환
            if not os.path.exists(image_path):
                missing_images.append(image_path)

    return missing_images

image_dir = '/content/yolov7/datasets/train/images'  # 이미지 파일 경로
label_dir = '/content/yolov7/datasets/train/labels'  # 라벨 파일 경로

missing_images = check_dataset_images(image_dir, label_dir)
if missing_images:
    print("Missing image files:")
    for img in missing_images:
        print(img)
else:
    print("All image files are present.")


All image files are present.


In [ ]:
import os

# 문제가 되는 파일 목록
problem_files = [
    'datasets/train/images/-Blvd-Antonio-l-Rodriguez-188-Conference-Room-Member-Engagement-1-1440x810_jpg.rf.5805433b140a16df868b05fd6c08a3f9.jpg',
    'datasets/train/images/0090099095a570d0_jpg.rf.e4a264724303fa097cae0e7661cc3ff7.jpg',
    'datasets/train/images/0383841103ca54df_jpg.rf.ebe532f73d5df430d7311ede99801eda.jpg',
    'datasets/train/images/01fbb47ed3360107_jpg.rf.27f52fad7e1462f39634c5c8fe073716.jpg',
    'datasets/valid/images/PartB_00635_jpg.rf.4ab4ee669c2d926cf7446ca896e8c280.jpg',
    'datasets/valid/images/PartB_00290_jpg.rf.d87f791b32603d5e8ebe01fb9e6b490c.jpg',
    'datasets/valid/images/PartB_01009_jpg.rf.9ffc990828a3f3c5ed9324159b9c474d.jpg',
    'datasets/valid/images/PartB_00089_jpg.rf.0066797c3289d722fe1d17fc8e0e3b49.jpg',
    'datasets/train/images/04oGNFFyHwyTgNIryLP66xs-8-v1645801745_jpg.rf.551a9c893c98068fac5f8431679b1475.jpg'
]

# 이미지 및 라벨 파일 제거 함수
def remove_files(problem_files):
    for file in problem_files:
        label_file = file.replace('images', 'labels').replace('.jpg', '.txt')

        # 이미지 파일 제거
        if os.path.exists(file):
            os.remove(file)
            print(f"Removed image file: {file}")
        else:
            print(f"Image file not found: {file}")

        # 라벨 파일 제거
        if os.path.exists(label_file):
            os.remove(label_file)
            print(f"Removed label file: {label_file}")
        else:
            print(f"Label file not found: {label_file}")

remove_files(problem_files)


Image file not found: datasets/train/images/-Blvd-Antonio-l-Rodriguez-188-Conference-Room-Member-Engagement-1-1440x810_jpg.rf.5805433b140a16df868b05fd6c08a3f9.jpg
Label file not found: datasets/train/labels/-Blvd-Antonio-l-Rodriguez-188-Conference-Room-Member-Engagement-1-1440x810_jpg.rf.5805433b140a16df868b05fd6c08a3f9.txt
Image file not found: datasets/train/images/0090099095a570d0_jpg.rf.e4a264724303fa097cae0e7661cc3ff7.jpg
Label file not found: datasets/train/labels/0090099095a570d0_jpg.rf.e4a264724303fa097cae0e7661cc3ff7.txt
Image file not found: datasets/train/images/0383841103ca54df_jpg.rf.ebe532f73d5df430d7311ede99801eda.jpg
Label file not found: datasets/train/labels/0383841103ca54df_jpg.rf.ebe532f73d5df430d7311ede99801eda.txt
Image file not found: datasets/train/images/01fbb47ed3360107_jpg.rf.27f52fad7e1462f39634c5c8fe073716.jpg
Label file not found: datasets/train/labels/01fbb47ed3360107_jpg.rf.27f52fad7e1462f39634c5c8fe073716.txt
Image file not found: datasets/valid/images/

In [ ]:
%cd /content/yolov7
!rm datasets/train/labels.cache
!rm datasets/valid/labels.cache


/content/yolov7


In [ ]:
import os

def check_dataset_images(image_dir, label_dir):
    missing_images = []

    for label_file in os.listdir(label_dir):
        if label_file.endswith('.txt'):
            image_file = label_file.replace('.txt', '.jpg')
            image_path = os.path.join(image_dir, image_file)
            image_path = image_path.replace("\\", "/")  # 경로 구분자 변환
            if not os.path.exists(image_path):
                missing_images.append(image_path)

    return missing_images

image_dir = '/content/yolov7/datasets/train/images'  # 이미지 파일 경로
label_dir = '/content/yolov7/datasets/train/labels'  # 라벨 파일 경로

missing_images = check_dataset_images(image_dir, label_dir)
if missing_images:
    print("Missing image files:")
    for img in missing_images:
        print(img)
else:
    print("All image files are present.")


All image files are present.


# Training Yolov7-tiny

In [ ]:
# YOLOv7 디렉토리로 이동
%cd /content/yolov7

# CPU에서 훈련 실행
!python train.py --img 416 --batch 8 --epochs 50 --data datasets/data.yaml --cfg cfg/training/yolov7-tiny.yaml --weights yolov7-tiny.pt --device 0



/content/yolov7
2024-06-04 02:33:26.980789: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-04 02:33:26.980842: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-04 02:33:26.982290: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-04 02:33:26.989697: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 02:33:28.132098: W

In [ ]:
# YOLOv7 디렉토리로 이동
%cd /content/yolov7
!python train.py --img 416 --batch 8 --epochs 13 --data datasets/data.yaml --cfg cfg/training/yolov7-tiny.yaml --weights 'yolov7-tiny.pt' --device 0


/content/yolov7
2024-06-04 06:02:23.694124: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-04 06:02:23.694199: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-04 06:02:23.696115: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-04 06:02:23.707006: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 06:02:25.330652: W

# Test

In [ ]:
!python test.py --weights runs/train/exp5/weights/best.pt --data datasets/data.yaml --img-size 640 --batch-size 16 --device 0

Namespace(weights=['runs/train/exp5/weights/best.pt'], data='datasets/data.yaml', batch_size=16, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 v0.1-128-ga207844 torch 2.3.0+cu121 CUDA:0 (Tesla T4, 15102.0625MB)

Fusing layers... 
IDetect.fuse
/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 208 layers, 6010302 parameters, 0 gradients, 13.0 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

val: Scanning 'datasets/valid/labels.cache' images and 

In [22]:
!zip -r /content/yolov7_ppdection_done.zip /content/yolov7

# 압축된 파일 다운로드
from google.colab import files

files.download('/content/yolov7_ppdection_done.zip')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  adding: content/yolov7/datasets/train/labels/photo-1461748659110-16121c049d52_jpg.rf.a90148a7b0109321bf8f2ea0ae0d493e.txt (deflated 31%)
  adding: content/yolov7/datasets/train/labels/images25_jpg.rf.2ecbd72f86743bc156682ed48298d1e7.txt (deflated 24%)
  adding: content/yolov7/datasets/train/labels/cell01_0013_jpg.rf.7e73a69d3f9d1a358b48a18de161ca54.txt (deflated 34%)
  adding: content/yolov7/datasets/train/labels/photo-1613566028327-4a32e02f6ab1_jpg.rf.a376b36954b132bd4dccdcfd49411780.txt (deflated 7%)
  adding: content/yolov7/datasets/train/labels/photo-1488510794195-3ab18be82210_jpg.rf.16926ff62474407d6b80888d03573d70.txt (deflated 32%)
  adding: content/yolov7/datasets/train/labels/photo-1650567695212-21b4345bd72c_jpg.rf.d0d143a3ff940be69f127e6bf9beaa7e.txt (deflated 52%)
  adding: content/yolov7/datasets/train/labels/photo-1519829424065-9fa2f0599299_jpg.rf.405ac30d00a560b2332d5147cc0f3397.txt (deflated 22%)
  adding: content/yolov7/datasets/tra

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>